# Project Notebook I: Data Pre-processing

David Chen

In this notebook, we will accomplish the following:

* Load clinical data as a DataFrame `clinical`, required to define class label `clinical[Label] : bool`
* Load molecular data as an mxn DataFrame, `genes`. We will select the appropriate rows (patients) and match the `genes` DataFrame with the `clinical` data.

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sagemaker
from sagemaker import get_execution_role

In [2]:
## My custom helper functions
from helpers import *

## Step 0. Load Sagemaker Resources

The data downloaded has been uploaded to S3 default bucket manually.

Note: These Jupyter Notebooks and associated S3 objects were on my personal AWS account.

In [3]:
sagemaker_session = sagemaker.Session()
sagemaker_session

In [4]:
role = sagemaker.get_execution_role()
role

'arn:aws:iam::644307290749:role/service-role/AmazonSageMaker-ExecutionRole-20210527T073969'

In [5]:
bucket = sagemaker_session.default_bucket()
bucket

'sagemaker-us-west-1-644307290749'

In [6]:
reveal_bucket(bucket)

'assets/'

## Step 1. Load & Handle Gene Features 

Gene features and clinical annotations (required for defining class label) are uploaded to my S3 buckets. 

The following cells use one of my helper functions to identify the path, and then import each as Pandas dataframes.

In [7]:
## Use my custom helper function imported:
path_input = get_s3_uri("assets/data/data_RNA_Seq_v2_mRNA_median_all_sample_Zscores.txt", bucket)
path_annot = get_s3_uri("assets/data/brca_tcga_clinical_data.tsv", bucket)

In [8]:
## Features - high dimensional
genes = pd.read_csv(path_input, delimiter="\t")
genes.shape

(20531, 1102)

In [9]:
genes.head()

,Hugo_Symbol,Entrez_Gene_Id,TCGA-3C-AAAU-01,TCGA-3C-AALI-01,TCGA-3C-AALJ-01,TCGA-3C-AALK-01,TCGA-4H-AAAK-01,TCGA-5L-AAT0-01,TCGA-5L-AAT1-01,TCGA-5T-A9QA-01,...,TCGA-UL-AAZ6-01,TCGA-UU-A93S-01,TCGA-V7-A7HQ-01,TCGA-W8-A86G-01,TCGA-WT-AB41-01,TCGA-WT-AB44-01,TCGA-XX-A899-01,TCGA-XX-A89A-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01
0,LOC100130426,100130426,-1.7608,-1.7608,1.1240,-1.7608,-1.7608,-1.7608,-1.7608,-1.7608,...,-1.7608,-1.7608,-1.7608,-1.7608,-1.7608,-1.7608,-1.7608,-1.7608,-1.7608,-1.7608
1,UBE2Q2P3,100133144,1.0944,0.3540,0.6451,0.6963,-0.0246,-0.6627,-2.9270,-1.6563,...,-2.4538,-0.5741,-2.9270,-0.2765,-2.9270,-2.9270,0.9240,1.5101,-1.2605,-0.0175
2,UBE2Q2P3,100134869,1.0230,1.4736,0.5206,0.7910,1.1891,1.1002,0.7244,0.4830,...,1.1830,0.7643,-0.9748,1.1829,-0.0262,-0.9051,1.7984,2.1743,0.1793,2.0240
3,LOC149767,10357,-1.7033,-1.0056,0.9114,0.7418,-0.5522,0.1842,0.2208,0.0383,...,0.5990,0.6079,-3.0182,0.4649,-1.6364,-2.2051,0.6371,-1.2071,-0.5285,1.1111
4,TIMM23,10431,-2.5248,-1.6307,0.8069,-0.4455,-0.7821,-0.6367,0.2389,0.9257,...,1.2029,2.8887,1.4583,-1.0420,0.5612,1.6977,-1.2260,-0.9444,0.3174,-0.0360


In [10]:
## Set aside gene name to numerical id
## Will be exported for reuse 
gene_map = genes.iloc[:, 0:2]

### Important: Drop _ESR1_ (estrogen receptor), _PGR_ (progesterone receptor), _ERBB2_ (HER2 receptor) genes to prevent data leakage

The specific reason as to why these 3 genes must be dropped to avoid _data leakage_ requires **domain-specific knowledge**, which is briefed in the _Proposal_ document.

In [11]:
## First check if the 3 genes are in the data
GENES_TO_DROP = ["ESR1", "PGR", "ERBB2"]
[g in genes["Hugo_Symbol"].values for g in GENES_TO_DROP]

[True, True, True]

In [12]:
## Next, drop the 3 genes from the dataframe 
genes = genes.loc[~genes["Hugo_Symbol"].isin(GENES_TO_DROP), :]

In [13]:
## Check: all should be False
[g in genes["Hugo_Symbol"].values for g in GENES_TO_DROP]

[False, False, False]

### Drop the gene names & gene code columns, which have been set aside

Keep ONLY the numerical gene IDs as row indices, which will become column indices 

In [14]:
assert not any(genes.duplicated("Entrez_Gene_Id"))  #ensure no duplicate id
genes.index = genes["Entrez_Gene_Id"]
genes = genes.iloc[:, 2:] #drop gene id cols

Note that the default format of the data on the web is that:

* Features (genes) are rows
* Observations (patients) are columns

This isn't what Python machine learning algorithms are typically expecting. Thus, we need to **transpose** the DataFrame such that:

* Features are columns
* Observations are rows

In [15]:
## Transpose dataframe such that each row=patient(observation), each col=gene(feature)
genes = genes.transpose()
genes.head() #preview again

Entrez_Gene_Id,100130426,100133144,100134869,10357,10431,136542,155060,26823,280660,317712,...,7789,158586,79364,440590,79699,7791,23140,26009,387590,389932
TCGA-3C-AAAU-01,-1.7608,1.0944,1.0230,-1.7033,-2.5248,NaN,2.6672,-1.9754,-1.0239,-3.8503,...,1.6674,1.3652,1.7323,1.1787,1.0112,0.0077,1.2480,0.7011,-2.2857,-1.0151
TCGA-3C-AALI-01,-1.7608,0.3540,1.4736,-1.0056,-1.6307,NaN,1.4841,0.6553,-0.5570,-3.8503,...,0.0469,-0.2845,0.8841,0.9389,-0.8990,0.9335,0.3568,-1.8110,1.7416,-1.0151
TCGA-3C-AALJ-01,1.1240,0.6451,0.5206,0.9114,0.8069,NaN,1.6791,-1.9754,-1.0239,-3.8503,...,-1.0439,0.0704,-1.3588,1.4057,-1.2453,0.9163,-0.4675,-1.2802,-0.4957,-1.0151
TCGA-3C-AALK-01,-1.7608,0.6963,0.7910,0.7418,-0.4455,NaN,1.5817,-0.7379,-1.0239,-3.8503,...,-0.1270,-0.3346,-0.8902,0.8269,-0.8818,1.0020,-0.9299,-0.5286,-0.0672,-1.0151
TCGA-4H-AAAK-01,-1.7608,-0.0246,1.1891,-0.5522,-0.7821,NaN,0.5166,-0.7082,-1.0239,-3.8503,...,-0.3710,-0.3982,-0.1067,-1.4182,-0.2086,0.2990,-0.7752,0.0127,0.6400,-1.0151


## Step 3. Load & Inspect Clinical Data, and Define Class Label

The clinical data is a separate, structured dataframe (text file) with columns necessary to define the "triple-negative" class label.

In [16]:
clinical = pd.read_csv(path_annot, delimiter="\t")
# sorted(clinical.columns.to_list())
clinical.shape

(1108, 140)

In [17]:
COLUMNS_CLIN = [
    'Sample ID', #ids
    'Sample Type','Fraction Genome Altered', #tumor info
    'Diagnosis Age','Sex','Race Category','Ethnicity Category', #patient info
    'Informed consent verified', #ethics
    'ER Status By IHC','PR status by ihc','IHC-HER2' #required for CLASS LABEL
]

clinical = clinical[COLUMNS_CLIN]
clinical.head()

,Sample ID,Sample Type,Fraction Genome Altered,Diagnosis Age,Sex,Race Category,Ethnicity Category,Informed consent verified,ER Status By IHC,PR status by ihc,IHC-HER2
0,TCGA-3C-AAAU-01,Primary,0.7787,55.0,Female,WHITE,NOT HISPANIC OR LATINO,YES,Positive,Positive,Negative
1,TCGA-3C-AALI-01,Primary,0.7164,50.0,Female,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,YES,Positive,Positive,Positive
2,TCGA-3C-AALJ-01,Primary,0.5340,62.0,Female,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,YES,Positive,Positive,Indeterminate
3,TCGA-3C-AALK-01,Primary,0.0764,52.0,Female,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,YES,Positive,Positive,Positive
4,TCGA-4H-AAAK-01,Primary,0.2364,50.0,Female,WHITE,NOT HISPANIC OR LATINO,YES,Positive,Positive,Equivocal


### Subset data & re-group features

We want to focus on:

* Primary, non-metastatic tumors (Metastasis can be a confounding variable and such tumors are very few in number)
* Female breast cancer patients only (There are <10 male patients, and gender is also a known confounding variable)

In [18]:
clinical = clinical[(clinical["Sex"]=="Female") & (clinical["Sample Type"]=="Primary")]

Aggregate Race Category into "white" vs. "non-white". Most patients in the dataset are white. This is a known limitation of the dataset, and has been critiqued in literature (hopefully in the future we will have datasets that cover a wider range of race).

In [19]:
print("Number of NaNs: %d" % sum(clinical['Race Category'].isna()))
clinical['RaceWhite'] = (clinical['Race Category']=="WHITE") 

Number of NaNs: 94


In [20]:
clinical.loc[clinical['Race Category'].isna(), "RaceWhite"] = None
sum(clinical['RaceWhite'].isna()) #should be same as above

94

### Define class labels, 1 if a cancer is ER-negative, PR-negative, and HER2-negative; 0 otherwise

Receptors ER, PR, HER2 have missing values!

* All 3 needs to be 'Negative' for Label=1. 
* As long as one of which is 'Positive' or 'Intermediate' or 'Equivocal', we have Label=0.
* Label=None is defined
    - All 3 are 'Negative'
    - 1 'Negative' + 2 NaN
    - 2 'Negative' + 1 NaN

We need to exclude samples whose Label=NaN.

In [21]:
## The last 3 integers indicate missing receptors required to define labels
[sum(clinical[col].isna()) for col in COLUMNS_CLIN] 

[0, 0, 18, 1, 0, 94, 171, 0, 49, 50, 176]

In [22]:
## Initialize the label definition:
clinical['Label'] = (clinical['ER Status By IHC']=='Negative') & \
                    (clinical['PR status by ihc']=='Negative') & \
                    (clinical['IHC-HER2']=='Negative')

In [23]:
clinical.loc[:,['ER Status By IHC','PR status by ihc','IHC-HER2']].isna()

,ER Status By IHC,PR status by ihc,IHC-HER2
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
1103,False,False,False
1104,False,False,False
1105,False,False,False
1106,False,False,False


In [24]:
## Count the number of missing receptors in each patient or sample
clinical['num_receptor_missing'] = clinical.loc[:,['ER Status By IHC','PR status by ihc','IHC-HER2']].isna().sum(axis=1)
set(clinical['num_receptor_missing'].to_list()) #few as 0, up to all 3 missing

{0, 1, 2, 3}

In [25]:
## Begin by setting any patient with 1+ receptor missing has having Label=None
clinical.loc[clinical['num_receptor_missing']>0, "Label"] = None
sum(clinical['Label'].isna())

178

In [26]:
## Then, re-set Label=0 from NaN if at least 1 receptor is NOT negative
label0_indicator = ['Positive','Intermediate','Equivocal']

clinical.loc[clinical['ER Status By IHC'].isin(label0_indicator) | \
         clinical['PR status by ihc'].isin(label0_indicator) | \
         clinical['IHC-HER2'].isin(label0_indicator), "Label"] = 0

sum(clinical['Label'].isna())

83

In [27]:
## Finally, exclude label-less rows (patients)
clinical = clinical.loc[~clinical['Label'].isna(), :]

### After defining our labels, make a summary table stratified by _Class Label=1 (True)_ vs. _Class Label=0 (False)_

We will use a "Table One" commonly used in medical and healthcare literature to summarize the dataset or patient population. Table One can be created for each feature one-by-one, but can also be automated by Python package `tableone`.

In [28]:
!pip install tableone

In [30]:
from tableone import TableOne

selected_cols = ['Sex','RaceWhite','num_receptor_missing','Informed consent verified']

myTable1 = TableOne(
    clinical,
    columns = selected_cols, 
    groupby = 'Label'
)

In [31]:
## Column 'True' or '1.0' means Class=1, i.e. the triple negative cancers
print(myTable1.tabulate(tablefmt="rst"))

================================  ======  =========  ============  ===========  ===========
..                                        Missing    Overall       0.0          1.0
================================  ======  =========  ============  ===========  ===========
n                                                    1002          886          116
Sex, n (%)                        Female  0          1002 (100.0)  886 (100.0)  116 (100.0)
RaceWhite, n (%)                  0.0     94         216 (23.8)    176 (22.0)   40 (36.7)
..                                1.0                692 (76.2)    623 (78.0)   69 (63.3)
num_receptor_missing, n (%)       0       0          907 (90.5)    791 (89.3)   116 (100.0)
..                                1                  95 (9.5)      95 (10.7)
Informed consent verified, n (%)  YES     0          1002 (100.0)  886 (100.0)  116 (100.0)
================================  ======  =========  ============  ===========  ===========


### Important note about research ethics

Even though the data is **publicly available**, the Table One above shows that 100\% data points have informed consent, a basic ethical principle for Healthcare research.

## Step 4. Mutually Subset Features DataFrame & Labels (Clinical) DataFrame for Export

* The exported data will be the data we use for Machine Learning tasks.
* Ensure all rows in the dataframe `genes` have a matching record in `clinical` (with class labels)
* Make the order of data and class label the same
* Export the subsetted, order-matched data to another S3 bucket folder `processed_data/` (I have manually created this folder in S3, under the parent directory `assets/`)

In [32]:
sum(clinical['Sample ID'].isin(genes.index)) #number of observations in each dataframe

999

In [33]:
sum(genes.index.isin(clinical['Sample ID'])) #should match cell above

999

In [34]:
clinical = clinical.loc[clinical['Sample ID'].isin(genes.index), :]
clinical.shape

(999, 14)

In [35]:
genes = genes.loc[clinical['Sample ID'],:]
genes.shape

(999, 20528)

In [36]:
## Bonus: Confirm mutually subsetted dataframes are also matched
all(genes.index == clinical['Sample ID'])

True

Export the processed data (features) and annotation (with class labels) to another directory in S3 bucket:

In [37]:
directS3Save(clinical, bucket, "assets/processed_data/clinical.csv")

In [38]:
directS3Save(genes, bucket, "assets/processed_data/genes.csv")

In [39]:
directS3Save(gene_map, bucket, "assets/processed_data/gene_name_map.csv")